# Image Preprocessing et Classification CNN
## Impact du Prétraitement sur la Reconnaissance de Panneaux Routiers

---

### 🎯 Objectifs Pédagogiques

Dans ce TP, vous allez :
1. **Comprendre** comment le prétraitement d'images affecte les performances d'un modèle CNN
2. **Expérimenter** avec différentes techniques de filtrage du bruit (P1)
3. **Explorer** les méthodes de correction de luminosité (P2)
4. **Analyser** pourquoi certains prétraitements améliorent la classification et d'autres la dégradent

### 📋 À propos du Dataset

- **GTSRB** : German Traffic Sign Recognition Benchmark
- **43 classes** de panneaux routiers
- Images de taille **64x64 pixels** en couleur (RGB)

### 📋 Prérequis

- Vous disposez du code du modèle CNN de base (P0_Model_Tensorflow.py)
- Le dataset GTSRB complet est disponible ici : https://www.kaggle.com/datasets/meowmeowmeowmeowmeow/gtsrb-german-traffic-sign 
- On travaillera avec un sous-ensemble  de 10% `./images/Train_small/` et des images de test `./images/Test/` 
- D'autres images "hors contexte" sont disponibles dans `./images/`
- Le fichier `utils.py` contient des fonctions utilitaires (affichage...etc)
- Le fichier `P0_Model_Tensorflow.py` contient le CNN (TensorFlow)

---

## 🔧 Configuration de l'Environnement

### ✨Première utilisation

Suivez ces étapes **avant** de lancer ce notebook :

1. **Ouvrir un terminal** dans le dossier du projet
2. **Créer l'environnement virtuel** :
   ```bash
   python -m venv .venv
   ```
3. **Activer l'environnement** :
   - Windows : `.venv\Scripts\activate`
   - Linux/Mac : `source .venv/bin/activate`
4. **Installer les dépendances** :
   ```bash
   pip install -r requirements.txt
   ```
5. **Enregistrer le kernel Jupyter** (pour que Jupyter utilise cet environnement) :
   ```bash
   python -m ipykernel install --user --name=icv-env --display-name "Python (ICV)"
   ```
6. **Lancer Jupyter** :
   ```bash
   jupyter notebook
   ```
7. Dans le menu du notebook (Visual Studio Code) : `Select Kernel` → `Jupyter Kernel...` → `Python (ICV)`

### 📦 Imports et Configuration

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from utils import show, add_noise
from P0_Model_Tensorflow import create_model_and_evaluate

# Configuration pour de meilleures visualisations
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("✓ Imports réussis")

✓ Imports réussis


---

## Partie 0 : Modèle de base (Sans Prétraitement)

### 🎯 Établir une Référence

Avant d'expérimenter avec différentes techniques de prétraitement, nous devons **établir une baseline** :
- Mesurer les performances du modèle CNN sur les images **brutes** (sans aucun traitement)
- Cette précision de référence nous permettra de **quantifier l'impact** de chaque technique de prétraitement

### 🏗️ Architecture du CNN

Le modèle de base (`P0_Model_Tensorflow.py`) utilise :
- 3 couches convolutionnelles (32 → 64 → 128 filtres)
- Max pooling après chaque convolution
- Couche dense de 128 neurones + Dropout (50%)
- Optimiseur Adam, fonction de perte categorical crossentropy

---

In [ ]:
# -------------------------------------------------------------------------------------------------
#
# Utiliser `create_model_and_evaluate` afin de déterminer la précision du modèle sans prétraitements
# Cette fonction :
#
# --------------------------------------------------------------------------------------------------


---

## Partie 1 : Filtrage du Bruit

### 🔍 Contexte Théorique

Le **bruit** dans les images peut provenir de :
- Capteurs de caméra (bruit thermique, photonique)
- Compression JPEG
- Transmission de données

### 🎧 Types de Bruit

Le bruit dégrade la qualité des images en ajoutant des variations indésirables aux valeurs des pixels. On distingue plusieurs types :

#### 1. Bruit Gaussien (Additif)
- **Modèle** : $I_{bruitée}(x,y) = I_{originale}(x,y) + N(0, \sigma^2)$
- **Caractéristiques** : Suit une distribution normale (courbe en cloche)
- **Origine** : Bruit électronique des capteurs, faible luminosité
- **Aspect visuel** : Grain uniforme sur toute l'image
- **Paramètres** : Moyenne (généralement 0) et écart-type $\sigma$

#### 2. Bruit Sel et Poivre (Impulsionnel)
- **Modèle** : Certains pixels prennent des valeurs extrêmes (0 = poivre noir, 255 = sel blanc)
- **Caractéristiques** : Pixels isolés très clairs ou très sombres
- **Origine** : Défauts de transmission, pixels morts du capteur
- **Aspect visuel** : Points blancs et noirs dispersés aléatoirement
- **Paramètre** : Densité de bruit (proportion de pixels affectés)

#### 3. Bruit de Poisson (Dépendant du Signal)
- **Modèle** : Variance proportionnelle à l'intensité du signal
- **Caractéristiques** : Plus visible dans les zones sombres
- **Origine** : Nature quantique de la lumière (comptage de photons)
- **Aspect visuel** : Grain variable selon l'intensité lumineuse

#### 4. Bruit Uniforme
- **Modèle** : Distribution uniforme sur un intervalle $[a, b]$
- **Origine** : Quantification, arrondis numériques
- **Moins courant** en pratique que les bruits gaussien et impulsionnel

### 🔇 Types de Filtres

#### Filtres Linéaires (filtres convolutifs)
1. **Filtre Moyenne** : Moyenne simple des pixels voisins
2. **Filtre Gaussien** : Moyenne pondérée avec poids gaussiens

#### Filtres Non Linéaires
3. **Filtre Médian** : Remplace chaque pixel par la médiane de son voisinage (robuste aux valeurs aberrantes)
4. **Filtre Bilatéral** : Lisse les zones homogènes tout en préservant les contours
5. **NLM (Non-Local Means)** : Exploite la redondance des patches dans toute l'image

### 📊 Métrique d'Évaluation : PSNR

Le **PSNR (Peak Signal-to-Noise Ratio)** mesure la qualité de reconstruction. C'est une mesure courament utilisée en compression pour évaluer la qualité de reconstruction. Elle peut aussi être utilisée pour quantifier la suppression de bruit, **dans les cas où l'on connait une *vérité terrain*** :
- Exprimé en décibels (dB)
- **Plus élevé = meilleur** (typiquement 20-40 dB)
- Formule : $PSNR = 20 \cdot \log_{10}\left(\frac{MAX}{\sqrt{MSE}}\right)$

où :
- **MAX** = Valeur maximale possible d'un pixel (255 pour des images 8 bits)
- **MSE** = Mean Squared Error (erreur quadratique moyenne) : $MSE = \frac{1}{N} \sum_{i=1}^{N} (I_{originale}(i) - I_{traitée}(i))^2$
  - $N$ = nombre total de pixels dans l'image
  - Plus le MSE est faible, plus les images sont similaires

**Interprétation** :
- PSNR > 40 dB : Excellente qualité, différences quasi imperceptibles
- 30-40 dB : Bonne qualité, légères différences visibles
- 20-30 dB : Qualité acceptable, différences notables
- < 20 dB : Faible qualité, dégradation importante

### 📊 Estimateur de bruit

Concerant notre dataset, nous ne connaissons pas de *vérité terrain* (si l'on suppose les images bruiées, nous ne connaissons pas les images d'origines correspondante). On peut cependant estimer la quantitié de bruit dans une image en mesurant la variance de son laplacien.

Sur notre image de test (lenna) nous allons tester différent filtre, mesurer le PSNR et la réduction de bruit avec notre estimateur, et vérifier que notre estimateur fonctionne bien.

---

### Ajout de Bruit Synthétique

In [ ]:
# ------------------------------------------------------------------------------------------------------
#
# Utiliser la fonction `add_noise` pour ajouer du bruit gaussien et 'salt and pepper' à l'image lenna.jpg
# Observer les 2 types de bruits séparément puis cumulés
# (Utiliser `show` pour visualiser les images)
#
# -------------------------------------------------------------------------------------------------------

### Implémentation des Filtres de Débruitage

In [ ]:
# -----------------------------------------------------------------------------------------------
#
# Appliquer les filtres suivants à l'image bruitée
# - **Moyenneur** : utiliser `cv2.blur()` avec un noyau 5×5
# - **Gaussien** : utiliser `cv2.GaussianBlur()` avec un noyau 5×5 et sigma=1
# - **Médian** : utiliser `cv2.medianBlur()` avec un noyau de taille 5
# - **Bilatéral** : utiliser `cv2.bilateralFilter()` avec d=9, sigmaColor=75, sigmaSpace=75
# - **NLM (Non-Local Means)** : utiliser `cv2.fastNlMeansDenoisingColored()` avec h=10, hColor=10
#
# Visualiser l'exécution pour chaque type de filtre
#
# -----------------------------------------------------------------------------------------------

    

### Évaluation Quantitative

In [ ]:
# -----------------------------------------------------------------------------------------
#
# Ecire une fonction qui calcule le PSNR entre une image débruitée et son originale
# Ecire une focntion qui estime le niveau de bruit dans une image (variance du laplacien)
#
# Appliquer ces 2 calculs aux différents filtres
#
# ------------------------------------------------------------------------------------------

   

### 💭 Questionnement ?

- Est ce que notre estimateur de bruit est correct ?
- Est ce que les valeurs de PSNR et de notre estimateur sont corrélées ?
- Est ce que les résultats donnés par le PSNR ou l'estimateur de bruit confirme ce que vous voyez visuellement ?
- Est ce qu'on va pouvoir se servir de l'estimateur pour estimer le bruit dans les images du dataset et choisir un filtre ?


---

### Application sur le Dataset GTSRB

In [ ]:
# ----------------------------------------------------------------------------------------------------
#
# Calculer le bruit moyen, minimun et maximum du dataset
# Isoler les images correspondantes
#
# Tester les différents filtres sur ces 3 images
# Observer visuellement les résultats données par le meilleur et moins bon au sens de notre estimateur. 
#
# ------------------------------------------------------------------------------------------------------


### ❓ Questionnement

- Les images du dataset GTSRB semblent-elles très bruitées ?
- Le filtrage du bruit est-il vraiment nécessaire pour ce dataset ?
- A-t-on intérêt à choisir le filtre le plus performant au sens de notre estimateur ?
- Ou plutôt un filtre qui préserve mieux les contour ?

---

### Création d'un Dataset Débruité et Évaluation CNN

In [ ]:
# ---------------------------------------------------------------------------------------------------------------------------
#
# Créer 2 dossiers `.image/Train_small_denoised` et `.image/Test_denoised` contenant les images filtrée avec le filtre retenu
#
# Recalculer le bruit moyen min et max sur le dossier .image/Train_small_denoised
#
# ----------------------------------------------------------------------------------------------------------------------------

### 🧪 Évaluation du Modèle sur Images Débruitées

In [ ]:
# -----------------------------------------
#
# Entrainer le CNN avec ce nouveau dataset
#
# -----------------------------------------


### 🤔 Analyse ?

- Pourquoi créer `.image/Test_denoised` ?
- Comment expliquer ces résultats ?

---

## Partie 2 : Correction de Luminosité

### 🌓 Contexte Théorique

Les images du dataset GTSRB présentent une **grande variabilité de luminosité** :
- Photos prises à différents moments de la journée (matin, midi, soir)
- Conditions météorologiques variées (ensoleillé, nuageux)
- Ombres projetées par des arbres, bâtiments, véhicules
- Caméras différentes avec expositions variables

### Techniques de Normalisation

#### 1. Égalisation d'Histogramme Classique
- Redistribue les intensités pour couvrir toute la plage [0, 255]
- **Avantage** : Simple et rapide
- **Inconvénient** : Peut sur-amplifier le bruit dans les zones homogènes ou supprimer du contraste localement

#### 2. CLAHE (Contrast Limited Adaptive Histogram Equalization)
- Égalisation locale par tuiles (tiles)
- Limitation du contraste (`clipLimit`) pour éviter la sur-amplification
- **Avantage** : Améliore le contraste tout en préservant les détails
- **Inconvénient** : Légèrement plus coûteux en calcul

### 🎨 Espace Colorimétrique YCrCb

On travaille dans l'espace **YCrCb** plutôt que RGB :
- **Y** : Luminance (information de luminosité)
- **Cr, Cb** : Chrominance (information de couleur)


La transformation RGB → YCrCb suit les formules standardisées **ITU-R BT.601** :

$$Y = 0.299 \cdot R + 0.587 \cdot G + 0.114 \cdot B$$

$$Cr = 128 + 0.500 \cdot (R - Y) = 128 + 0.500 \cdot R - 0.419 \cdot G - 0.081 \cdot B$$

$$Cb = 128 + 0.500 \cdot (B - Y) = 128 - 0.169 \cdot R - 0.331 \cdot G + 0.500 \cdot B$$

où $R$, $G$, $B \in [0, 255]$ et $Y$, $Cr$, $Cb \in [0, 255]$


Le canal Y correspond à une conversion en niveaux de gris mais avec une pondération différente sur les canaux RGB dérivée de la perception humaine :
- L'œil humain est plus sensible au vert (coefficient 0.587)
- Moyennement sensible au rouge** (0.299)
- Peu sensible au bleu** (0.114)

En changeant d'espace colorimétrique :

1. **Séparation luminosité/couleur** : En isolant Y, on peut traiter la luminosité indépendamment de la teinte
2. **Préservation des couleurs** : En modifiant uniquement Y, on normalise la luminosité tout en gardant les canaux Cr et Cb intacts → les couleurs originales sont préservées !

---

### Implémentation des Techniques de Correction

In [ ]:
# ------------------------------------------------------------------------------
#
# Ecrire 2 fonctions pour appliquer les corrections de luminosité sur le canal Y
# (utiliser cvtColor, equalizeHist, createCLAHE) 
#
# Visualiser les 2 corrections sur l'image 2under-exposed.jpg
#
# ------------------------------------------------------------------------------

### Analyse de la Variabilité de Luminosité dans le Dataset

In [ ]:
# ------------------------------------------------------------------------------------------------------
#
# Ecrire une focntion qui permet d'estimer la luminosité d'une image
# Comparer la distribution de la luminosité (histogramme) sur l'ensemble du dataset avant et après CLAHE
# observer visuellement l'effet de CLAHE sur l'image la plus sombre, la plus claire et de luminosité moyenne
#
# -------------------------------------------------------------------------------------------------------

### 💡 Observation Clé

Remarquez comment CLAHE **normalise** les trois images malgré leurs différences initiales de luminosité. Cela permet au CNN de se concentrer sur la **forme et le contenu** du panneau plutôt que sur les conditions d'éclairage.

---

## 2.5 - Création d'un Dataset Normalisé et Évaluation CNN

### 🧪 Évaluation du Modèle avec Correction de Luminosité

## ✅ Conclusions ?

---